# 自动处理越南语没有成功翻译的字段

In [3]:
pip install openai python-dotenv

  Using cached openai-1.58.1-py3-none-any.whl.metadata (27 kB)
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
  Using cached anyio-4.7.0-py3-none-any.whl.metadata (4.7 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached jiter-0.8.2-cp310-cp310-win_amd64.whl.metadata (5.3 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached httpcore-1.0.7-py3-none-any.whl.metadata (21 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
Using cached openai-1.58.1-py3-none-any.whl (454 kB)
Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)
Using cached anyio-4.7.0-py3-none-any.whl (93 kB)
Using cached distro-1.9.0-py3-none-any.whl (20 kB)
Using cached httpx-0.28.1-py3-none-any.whl (73 kB)
Using cached httpcore-1.0.7-py3-none-any.whl (78 kB)
Using cached jiter-0.8.2-cp310-cp310-win_amd64.whl (204 kB)
Using cached sniffio-1.3.1-py3-none-any.whl 


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
pip install openpyxl

  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached et_xmlfile-2.0.0-py3-none-any.whl.metadata (2.7 kB)
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
Using cached et_xmlfile-2.0.0-py3-none-any.whl (18 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import re

from openai import OpenAI
from dotenv import load_dotenv

def contains_chinese(text):
    # 使用正则表达式匹配中文字符
    return bool(re.search(r'[\u4e00-\u9fff]', text))

load_dotenv()
client = OpenAI()
original_language = "Chinese"
target_language = "Korean"
def generate_text(index, data):
    if data["module"] == "Error: Connection error.":
        completion = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": f"You are an expert in translating {original_language} to {target_language} for ERP systems. Your task is to translate markdown-formatted text from {original_language} to {target_language}. Preserving its formatting without adding extra content."},
                {"role": "user", "content": str(data['value'])}
            ],
            temperature=0
        )
        return index, completion.choices[0].message.content
    else:
        return index, str(data["module"])
    
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
import pandas as pd

filenames = [
    r"C:\Users\wangz\Downloads\翻译韩语从B到C列-0116.xlsx",
]
for filename in filenames:
    to_fix = pd.read_excel(filename)

    with ThreadPoolExecutor(max_workers=1000) as executor:
        futures = {executor.submit(generate_text, index, row) for index, row in to_fix.iterrows()}

        for future in tqdm(as_completed(futures), total=len(futures)):
            index, output = future.result()

            if output is not None:
                to_fix.at[index, "module"] = output
    # 保存最终结果到 Excel 文件
    to_fix.to_excel(filename.replace(".xlsx", "_translated.xlsx"), index=False)

100%|██████████| 190793/190793 [00:05<00:00, 33760.82it/s]


In [10]:
to_fix.to_csv(r"C:\Users\wangz\Downloads\未翻译词条_1.csv", index=False)

In [6]:
to_fix = pd.read_excel(r"C:\Users\wangz\Downloads\未翻译词条.xlsx")

# 第二次检测修复字符串

In [7]:
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()
client = OpenAI()
original_language = "English"
target_language = "Vietnamese"
def generate_text(index, data):
    try:
        if str(data['备注']).strip() == 'SPECIAL VALUE' or str(data['备注']).strip() == 'SPECIAL VALUE, INCLUDE UPPERCASE LETTER' or str(data['备注']).strip() == '':
            return index, str(data['待翻译(译)']).strip()
        if str(data['参考语言(英文)']).strip().lower() != str(data['简体中文(源)']).strip().lower() and str(data['参考语言(英文)']).strip().lower() == str(data['待翻译(译)']).strip().lower():
            completion = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": f"You are an expert in translating {original_language} to {target_language} for ERP systems. Your task is to translate markdown-formatted text from {original_language} to {target_language}. Preserving its formatting without adding extra content."},
                    {"role": "user", "content": str(data['参考语言(英文)'])}
                ],
                temperature=0
            )
            return index, completion.choices[0].message.content
        if str(data['参考语言(英文)']).strip().lower() == str(data['简体中文(源)']).strip().lower():
            return index, str(data['参考语言(英文)']).strip()
        else:
            completion = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "user", "content": "Translate English: The received L/G %s is not submitted \nTo Spanish: El L/G recibido no se ha enviado. \nPlease fix the translate error, because %s is missing, translate it directly and without adding any extra content."},
                    {"role": "assistant", "content": "El L/G recibido %s no se ha enviado."},
                    {"role": "user", "content": f"Translate {original_language}: {str(data['参考语言(英文)'])} \nTo {target_language}: {str(data['待翻译(译)'])} \n Please fix the translate error, Because {str(data['备注'])}. translate it directly and without adding any extra content."}
                ],
                temperature=0
            )
            return index, completion.choices[0].message.content
    except:
        return index, str(data['待翻译(译)']).strip()

In [8]:
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
import pandas as pd
to_fix = pd.read_excel(r"D:\Projects\ai-translator\special value\first_check\processed_files\multilangInitData YS全量词条 20241119 Special Value_中_越.xlsx")

with ThreadPoolExecutor(max_workers=1000) as executor:
    futures = {executor.submit(generate_text, index, row) for index, row in to_fix.iterrows()}

    for future in tqdm(as_completed(futures), total=len(futures)):
        index, output = future.result()

        if output is not None:
            to_fix.at[index, f'待翻译(译)'] = output

to_fix['备注'] = ""
# 保存最终结果到 Excel 文件
to_fix.to_excel(r"D:\Projects\ai-translator\special value\first_check\processed_files\multilangInitData YS全量词条 20241119 Special Value_中_越_processed.xlsx", index=False)

100%|██████████| 124202/124202 [02:26<00:00, 849.76it/s] 


# 文件合并

In [2]:
import pandas as pd

# 定义要合并的 Excel 文件列表
excel_files = [
    r"C:\Users\wangz\Downloads\multilangInitData20241211英语译葡萄牙语（翻译后）\part_1.xlsx",
    r"C:\Users\wangz\Downloads\multilangInitData20241211英语译葡萄牙语（翻译后）\part_2.xlsx",
    r"C:\Users\wangz\Downloads\multilangInitData20241211英语译葡萄牙语（翻译后）\part_3.xlsx",
    r"C:\Users\wangz\Downloads\multilangInitData20241211英语译葡萄牙语（翻译后）\part_4.xlsx",
    r"C:\Users\wangz\Downloads\multilangInitData20241211英语译葡萄牙语（翻译后）\part_5.xlsx",
    r"C:\Users\wangz\Downloads\multilangInitData20241211英语译葡萄牙语（翻译后）\part_6.xlsx",
    r"C:\Users\wangz\Downloads\multilangInitData20241211英语译葡萄牙语（翻译后）\part_7.xlsx",
    r"C:\Users\wangz\Downloads\multilangInitData20241211英语译葡萄牙语（翻译后）\part_8.xlsx",
    r"C:\Users\wangz\Downloads\multilangInitData20241211英语译葡萄牙语（翻译后）\part_9.xlsx",
    r"C:\Users\wangz\Downloads\multilangInitData20241211英语译葡萄牙语（翻译后）\part_10.xlsx",
    r"C:\Users\wangz\Downloads\multilangInitData20241211英语译葡萄牙语（翻译后）\part_11.xlsx",
]

# 读取并合并所有 Excel 文件
df_list = [pd.read_excel(file) for file in excel_files]
merged_df = pd.concat(df_list, ignore_index=True)

# 保存合并后的 DataFrame 到新的 Excel 文件
merged_df.to_excel(r"C:\Users\wangz\Downloads\multilangInitData20241211英语译葡萄牙语（翻译后）\multilangInitData20241211英语译葡萄牙语（翻译后）.xlsx", index=False)


# 文件拆分

In [1]:
import os
import pandas as pd

def split_excel(input_file, output_folder, chunk_size=100000):
    """
    将一个Excel文件按指定行数拆分为多个Excel文件，并存储到指定文件夹。

    Args:
        input_file (str): 输入的Excel文件路径。
        output_folder (str): 输出文件夹路径。
        chunk_size (int): 每个拆分Excel文件包含的最大行数，默认10万。
    """
    # 检查输出文件夹是否存在，不存在则创建
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # 读取Excel文件（假设默认是Sheet1）
    print("正在读取Excel文件...")
    excel_data = pd.read_excel(input_file, sheet_name=0)

    # 获取总行数
    total_rows = excel_data.shape[0]
    print(f"Excel文件总行数: {total_rows} 行")

    # 按chunk_size进行分割
    for i in range(0, total_rows, chunk_size):
        chunk = excel_data.iloc[i:i + chunk_size]  # 提取数据块
        output_file = os.path.join(output_folder, f"part_{i // chunk_size + 1}.xlsx")
        chunk.to_excel(output_file, index=False, engine='openpyxl')  # 保存为Excel
        print(f"已保存: {output_file}")
    
    print("拆分完成！")

# 示例用法
if __name__ == "__main__":
    input_file = r"C:\Users\wangz\Downloads\multilangInitData20241211空.xlsx"  # 输入文件路径
    output_folder = r"D:\Projects\ai-translator\src\multilangInitData20241211空"  # 输出文件夹路径
    split_excel(input_file, output_folder)


正在读取Excel文件...


d:\Anaconda\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Excel文件总行数: 1014546 行
已保存: D:\Projects\ai-translator\src\multilangInitData20241211空\part_1.xlsx
已保存: D:\Projects\ai-translator\src\multilangInitData20241211空\part_2.xlsx
已保存: D:\Projects\ai-translator\src\multilangInitData20241211空\part_3.xlsx
已保存: D:\Projects\ai-translator\src\multilangInitData20241211空\part_4.xlsx
已保存: D:\Projects\ai-translator\src\multilangInitData20241211空\part_5.xlsx
已保存: D:\Projects\ai-translator\src\multilangInitData20241211空\part_6.xlsx
已保存: D:\Projects\ai-translator\src\multilangInitData20241211空\part_7.xlsx
已保存: D:\Projects\ai-translator\src\multilangInitData20241211空\part_8.xlsx
已保存: D:\Projects\ai-translator\src\multilangInitData20241211空\part_9.xlsx
已保存: D:\Projects\ai-translator\src\multilangInitData20241211空\part_10.xlsx
已保存: D:\Projects\ai-translator\src\multilangInitData20241211空\part_11.xlsx
拆分完成！


# 友互通翻译

In [1]:
from openai import OpenAI
from dotenv import load_dotenv
import zhconv

load_dotenv()
client = OpenAI()
target_columns = [
    "印尼语",
    "匈牙利语",
    "葡萄牙语",
    "泰语",
    "土耳其语",
    "越南语",
    "俄语",
    "阿拉伯语",
    "芬兰语",
    "丹麦语",
    "荷兰语",
    "波兰语",
    "法语",
    "德语",
    "日语",
    "挪威语",
    "希伯来语",
    "韩语",
    "西班牙语",
    "捷克语",
    "意大利语",
    "瑞典语",
    "希腊语",
    "马来语",
    "斯洛伐克语",
    "柬埔寨语",
    "罗马尼亚语",
    "克罗地亚语",
    "乌兹别克语",
    "缅甸语"
]

# target_columns = [
#     "阿拉伯语", "捷克语", "丹麦语", "德语", "希腊语", "西班牙语", "芬兰语", "法语", "希伯来语", "克罗地亚语", "匈牙利语", "印度尼西亚语", "意大利语", "日语", "柬埔寨语", "韩语", "老挝语", "马来语", "缅甸语", "挪威语", "荷兰语", "波兰语", "葡萄牙语", "罗马尼亚语", "俄语", "斯洛伐克语", "瑞典语", "泰语", "土耳其语", "乌克兰语", "乌兹别克语", "越南语"
# ]

simple_column_name = "简体中文(源)"
english_column_name = "English"
trans_column_name = "繁体中文"

for target_column in target_columns:
    dictionary = {}

    def generate_text(index, data):
        if not pd.isnull(data[target_column]):
            dictionary[str(data[english_column_name])] = data[target_column]
            return index, data[target_column]
        if str(data[english_column_name]) in dictionary.keys():
            return index, dictionary[str(data[english_column_name])]
        if target_column == trans_column_name:
            return index, zhconv.convert(str(data[simple_column_name]), 'zh-tw')
        completion = client.chat.completions.create(
            model='gpt-4o',
            messages=[
                {"role": "user", "content": f"Translate the following sentence from English to {simple_column_name}: {str(data[english_column_name])}, please directly translate it and do not output any extra content"},
                {"role": "assistant", "content": str(data[simple_column_name])},
                {"role": "user", "content": f"Translate the following sentence from English to {trans_column_name}: {str(data[english_column_name])}, please directly translate it and do not output any extra content"},
                {"role": "assistant", "content": str(data[trans_column_name])},
                {"role": "user", "content": f"Translate the following sentence from English to {target_column}: {str(data[english_column_name])}, please directly translate it and do not output any extra content"}
            ],
            temperature=0
        )
        dictionary[str(data[english_column_name])] = completion.choices[0].message.content
        return index, completion.choices[0].message.content

    from tqdm import tqdm
    from concurrent.futures import ThreadPoolExecutor, as_completed
    import pandas as pd
    to_fix = pd.read_excel(r"C:\Users\wangz\Downloads\友户通-新增词条-20250114.xlsx")

    with ThreadPoolExecutor(max_workers=1000) as executor:
        futures = {executor.submit(generate_text, index, row) for index, row in to_fix.iterrows()}

        for future in tqdm(as_completed(futures), total=len(futures)):
            index, output = future.result()

            if output is not None:
                to_fix.at[index, target_column] = output

    # 保存最终结果到 Excel 文件
    to_fix.to_excel(r"C:\Users\wangz\Downloads\友户通-新增词条-20250114.xlsx", index=False)

  0%|          | 0/4 [00:00<?, ?it/s]C:\Users\wangz\AppData\Local\Temp\ipykernel_33656\1623887063.py:85: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Tips Keamanan' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  to_fix.at[index, target_column] = output
  0%|          | 0/4 [00:00<?, ?it/s]C:\Users\wangz\AppData\Local\Temp\ipykernel_33656\1623887063.py:85: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Biztonsági tippek' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  to_fix.at[index, target_column] = output
  0%|          | 0/4 [00:00<?, ?it/s]C:\Users\wangz\AppData\Local\Temp\ipykernel_33656\1623887063.py:85: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '

In [2]:
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()
client = OpenAI(base_url="https://generativelanguage.googleapis.com/v1beta/", api_key="")
target_column = "印尼语"

def generate_text(index, data):
    if not pd.isnull(data[target_column]):
        return index, data[target_column]
    completion = client.chat.completions.create(
        model='gemini-1.5-flash',
        messages=[
            {"role": "user", "content": f"Translate English to Chinese: {str(data['English(译)'])}, please directly translate it and do not output any extra content"},
            {"role": "assistant", "content": str(data['简体中文(源)'])},
            {"role": "user", "content": f"Translate English to Traditional Chinese: {str(data['English(译)'])}, please directly translate it and do not output any extra content"},
            {"role": "assistant", "content": str(data['繁体中文（译）'])},
            {"role": "user", "content": f"Translate English to {target_column}: {str(data['English(译)'])}, please directly translate it and do not output any extra content"}
        ],
        temperature=0
    )
    return index, completion.choices[0].message.content

from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
import pandas as pd
to_fix = pd.read_excel(r"D:\Projects\ai-translator\src\友互通\友互通-全.xlsx")

with ThreadPoolExecutor(max_workers=1000) as executor:
    futures = {executor.submit(generate_text, index, row) for index, row in to_fix.iterrows()}

    for future in tqdm(as_completed(futures), total=len(futures)):
        index, output = future.result()

        if output is not None:
            to_fix.at[index, target_column] = output

# 保存最终结果到 Excel 文件
to_fix.to_excel(r"D:\Projects\ai-translator\src\友互通\友互通-全.xlsx", index=False)

(0, 'matahari\n')

# 翻译123种语言

初翻

In [2]:
import pandas as pd
from openai import OpenAI
import os
from dotenv import load_dotenv
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import threading

# Load environment variables and set OpenAI API key
load_dotenv()
client = OpenAI()
# Read the Excel file into a pandas DataFrame
data = pd.read_excel(r"C:\Users\wangz\Downloads\20241203 123种语种翻译数据.xlsx")

# Define the columns and rows to process
columns = data.columns.to_list()[4:]  # Exclude the first 4 columns
rows = data.index[2:]  # Start from the third row

# Prepare the list of tasks for translation
tasks = []
for column in columns:
    for row_index in rows:
        text_to_translate = data.at[row_index, 'zh_CN']
        tasks.append((row_index, column, text_to_translate))

# Initialize a lock for thread-safe DataFrame operations
data_lock = threading.Lock()
max_workers = 2000  # Adjust the number of threads as needed

# Define the function to translate a single cell
def translate_cell(task):
    row_index, column, text_to_translate = task
    messages = [
        {"role": "user", "content": f"Translate '简体中文' to {column}, translate it directly and without adding any extra content."},
        {"role": "assistant", "content": str(data[column][1])},
        {"role": "user", "content": f"Translate '{text_to_translate}' to {column}, translate it directly and without adding any extra content."}
    ]
    try:
        completion =  client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            temperature=0
        )
        translated_text = completion.choices[0].message.content
        # Write the translated text back to the DataFrame in a thread-safe manner
        with data_lock:
            data.at[row_index, column] = translated_text
        return (row_index, column, translated_text)
    except Exception as e:
        print(f"Error translating row {row_index}, column {column}: {e}")
        return (row_index, column, None)

# Execute the translation tasks using a ThreadPoolExecutor and display a progress bar
with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = [executor.submit(translate_cell, task) for task in tasks]
    for future in tqdm(as_completed(futures), total=len(futures)):
        pass  # The results are handled within the translate_cell function

# Save the translated DataFrame to a new Excel file
data.to_excel("translated_data.xlsx", index=False)

100%|██████████| 14640/14640 [02:03<00:00, 118.87it/s]


In [1]:
import pandas as pd
from openai import OpenAI
import os
from dotenv import load_dotenv
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import threading

# Load environment variables and set OpenAI API key
load_dotenv()
client = OpenAI()
# Read the Excel file into a pandas DataFrame
data = pd.read_excel(r"C:\Users\wangz\Documents\WeChat Files\wxid_eet1drkwse4k22\FileStorage\File\2024-12\20241203 123种语种翻译数据-GPT 确认.xlsx")

# Define the columns and rows to process
columns = data.columns.to_list()[4:]  # Exclude the first 4 columns
rows = data.index[2:]  # Start from the third row

# Prepare the list of tasks for translation
tasks = []
for column in columns:
    for row_index in rows:
        text_to_translate = data.at[row_index, 'zh_CN']
        if text_to_translate == '挪威语':
            tasks.append((row_index, column, text_to_translate))
            break  # Only translate the first occurrence

# Initialize a lock for thread-safe DataFrame operations
data_lock = threading.Lock()
max_workers = 2000  # Adjust the number of threads as needed

# Define the function to translate a single cell
def translate_cell(task):
    row_index, column, text_to_translate = task
    messages = [
        {"role": "user", "content": f"Translate '简体中文' to {column}, translate it directly and without adding any extra content."},
        {"role": "assistant", "content": str(data[column][1])},
        {"role": "user", "content": f"Translate '{text_to_translate}' to {column}, translate it directly and without adding any extra content."}
    ]
    try:
        completion =  client.chat.completions.create(
            model="gpt-4o",
            messages=messages,
            temperature=0
        )
        translated_text = completion.choices[0].message.content
        # Write the translated text back to the DataFrame in a thread-safe manner
        with data_lock:
            data.at[row_index, column] = translated_text
        return (row_index, column, translated_text)
    except Exception as e:
        print(f"Error translating row {row_index}, column {column}: {e}")
        return (row_index, column, None)

# Execute the translation tasks using a ThreadPoolExecutor and display a progress bar
with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = [executor.submit(translate_cell, task) for task in tasks]
    for future in tqdm(as_completed(futures), total=len(futures)):
        pass  # The results are handled within the translate_cell function

# Save the translated DataFrame to a new Excel file
data.to_excel("translated_data.xlsx", index=False)

100%|██████████| 120/120 [00:03<00:00, 32.88it/s] 


精翻

In [6]:
import pandas as pd
from openai import OpenAI
import os
from dotenv import load_dotenv
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import threading

# Load environment variables and set OpenAI API key
load_dotenv()
client = OpenAI()
# Read the Excel file into a pandas DataFrame
data = pd.read_excel(r"D:\Projects\ai-translator\src\translated_data.xlsx")

# Define the columns and rows to process
columns = data.columns.to_list()[4:]  # Exclude the first 4 columns
rows = data.index[2:]  # Start from the third row

# Prepare the list of tasks for translation
tasks = []
for column in columns:
    for row_index in rows:
        text_to_translate = data.at[row_index, 'zh_CN']
        tasks.append((row_index, column, text_to_translate))

# Initialize a lock for thread-safe DataFrame operations
data_lock = threading.Lock()
max_workers = 2000  # Adjust the number of threads as needed

# Define the function to translate a single cell
def translate_cell(task):
    row_index, column, text_to_translate = task
    if text_to_translate.strip() == data.at[row_index, column].strip():
        messages = [
            {"role": "user", "content": f"Translate '简体中文' to {column}, translate it directly and without adding any extra content."},
            {"role": "assistant", "content": str(data[column][1])},
            {"role": "user", "content": f"Translate '{text_to_translate}' to {column}, translate it directly and without adding any extra content."}
        ]
        try:
            completion =  client.chat.completions.create(
                model="gpt-4o",
                messages=messages,
                temperature=0
            )
            translated_text = completion.choices[0].message.content
            # Write the translated text back to the DataFrame in a thread-safe manner
            with data_lock:
                data.at[row_index, column] = translated_text
            return (row_index, column, translated_text)
        except Exception as e:
            print(f"Error translating row {row_index}, column {column}: {e}")
            return (row_index, column, None)
    else:
        return (row_index, column, data.at[row_index, column])

# Execute the translation tasks using a ThreadPoolExecutor and display a progress bar
with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = [executor.submit(translate_cell, task) for task in tasks]
    for future in tqdm(as_completed(futures), total=len(futures)):
        pass  # The results are handled within the translate_cell function

# Save the translated DataFrame to a new Excel file
data.to_excel("translated_data_1.xlsx", index=False)

100%|██████████| 14640/14640 [00:01<00:00, 14606.36it/s] 


# 统计文件夹下的文件数

In [14]:
import os

def count_files(folder_path):
    total_files = 0
    for root, dirs, files in os.walk(folder_path):
        total_files += len(files)  # 统计当前目录的文件数量
    return total_files

# 输入文件夹路径
folder_path = r"C:\Users\wangz\Documents\WeChat Files\wxid_eet1drkwse4k22\FileStorage\File\2024-12\高级版2312帮助文档（English）\高级版2312帮助文档（English）"
file_count = count_files(folder_path)
print(f"文件总数为: {file_count}")


文件总数为: 5001


# 翻译越南语

In [ ]:
from openai import OpenAI
from dotenv import load_dotenv
import zhconv

load_dotenv()
client = OpenAI()
target_columns = [
    "目标语言"
]

for target_column in target_columns:
    dictionary = {}

    def generate_text(index, data):
        if not pd.isnull(data[target_column]):
            dictionary[str(data['English(译)'])] = data[target_column]
            return index, data[target_column]
        if str(data['English(译)']) in dictionary.keys():
            return index, dictionary[str(data['English(译)'])]
        if target_column == "繁体中文（译）":
            return index, zhconv.convert(str(data['简体中文(源)']), 'zh-tw')
        completion = client.chat.completions.create(
            model='gpt-4o-mini',
            messages=[
                {"role": "user", "content": f"Translate English to Chinese: {str(data['English(译)'])}, please directly translate it and do not output any extra content"},
                {"role": "assistant", "content": str(data['简体中文(源)'])},
                {"role": "user", "content": f"Translate English to Traditional Chinese: {str(data['English(译)'])}, please directly translate it and do not output any extra content"},
                {"role": "assistant", "content": str(data["繁体中文（译）"])},
                {"role": "user", "content": f"Translate English to {target_column}: {str(data['English(译)'])}, please directly translate it and do not output any extra content"}
            ],
            temperature=0
        )
        dictionary[str(data['English(译)'])] = completion.choices[0].message.content
        return index, completion.choices[0].message.content

    from tqdm import tqdm
    from concurrent.futures import ThreadPoolExecutor, as_completed
    import pandas as pd
    to_fix = pd.read_excel(r"C:\Users\wangz\Downloads\前端词条补充翻译-20241227\TINPERNEXTPRO-FE_en_US20241226203251.xlsx")

    with ThreadPoolExecutor(max_workers=1000) as executor:
        futures = {executor.submit(generate_text, index, row) for index, row in to_fix.iterrows()}

        for future in tqdm(as_completed(futures), total=len(futures)):
            index, output = future.result()

            if output is not None:
                to_fix.at[index, target_column] = output

    # 保存最终结果到 Excel 文件
    to_fix.to_excel(r"C:\Users\wangz\Downloads\前端词条补充翻译-20241227\TINPERNEXTPRO-FE_en_US20241226203251.xlsx", index=False)